In [9]:
import c3ltc
import visualize_c3ltc

In [10]:
G = PSL(2,7)
C_a = ReedSolomonCode(GF(7), Integer(6), Integer(4))
C_b = ReedSolomonCode(GF(7), Integer(6), Integer(4))
A = random_generators(G,6)
B = random_generators(G,6)
c3ltc = c3LTC(C_a, C_b, G, A,B)
word = c3ltc.decode(vector(GF(7), [0, 2]+[0]*1510))
numpy.count_nonzero(c3ltc.syndrome(word) ) == 0

True

In [11]:
show_square(c3ltc,1)

,vertex,left,right,closing
1,95,40,116,40


In [12]:
local_view(c3ltc,1)

,60,39,56,5,79,13
55,18,19,20,21,22,23
80,1101,1014,1102,1103,1088,1104
39,24,25,26,27,28,29
60,620,24,580,885,886,887
126,30,31,32,33,34,35
136,524,1417,202,1457,1454,700


In [13]:
local_view_in_word(c3ltc,word,1)

,60,39,56,5,79,13
55,0,0,0,0,0,0
80,0,0,0,0,0,0
39,0,0,0,0,0,0
60,0,0,0,0,0,0
126,0,0,0,0,0,0
136,0,0,0,0,0,0


In [14]:
show_common_views_in_word(c3ltc,word,1,c3ltc.vertex_to_neighbours_left[1][0],"left")

,60,39,56,5,79,13
55,0,0,0,0,0,0
80,0,0,0,0,0,0
39,0,0,0,0,0,0
60,0,0,0,0,0,0
126,0,0,0,0,0,0
136,0,0,0,0,0,0
,9,20,93,134,77,129
80,0,0,0,0,0,0
1,0,0,0,0,0,0
144,0,0,0,0,0,0
